#### Description

In [1]:
print('Author:  Leo Pauly (cnlp@leeds.ac.uk) & Nick Wilson (n.wilson@lubs.leeds.ac.uk)')
print('Description: Autmatic database update: Conversion')

Author:  Leo Pauly (cnlp@leeds.ac.uk) & Nick Wilson (n.wilson@lubs.leeds.ac.uk)
Description: Autmatic database update: Conversion


#### Usage intructions & Other info

1. Change/Add User (user variable options: 'leo','nick')
2. Change file_formate variable (options: '.sav','.csv')
3. Prefefably install python using anacodna (all in one installation): https://www.anaconda.com/products/individual#windows
4. Run this if 'pyarrow' module is missing':  !pip install pyarrow 
5. Converted database are stored in the directory : UKLTD_Database 

#### Imports

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import pyreadstat
from zipfile import ZipFile
import pyunpack
import multiprocessing
import dask.dataframe as dd
import time
import dask
from dask.diagnostics import ProgressBar
from dask.distributed import Client
dask.config.set(scheduler='threads')
import gc

print('Python version:',sys.version)
num_processes = multiprocessing.cpu_count()
print('No: of logical CPU cores available:',num_processes)

AC01_header_dtypes={'AC01': 'O', 'REGNUM': 'O','ACCDAT_start': 'O','ACCDAT': 'O', 'number_of_weeks':'float64', 'months':'float64', 'currency': 'O', 'consolidated': 'O', 'acctype': 'float64', 'Turnover': 'float64', 'Export': 'float64', 'Cost_of_Sales': 'float64', 'Gross_Profit': 'float64', 'Wages_Salaries': 'float64', 'Directors_Emoluments': 'float64', 'Operating_Profits': 'float64', 'Depreciation': 'float64', 'Audit_Fees': 'float64', 'Interest_Payments': 'float64', 'Pre_Tax_Profit': 'float64', 'taxation1': 'float64', 'Profit_After_Tax': 'float64', 'Dividends_Payable': 'float64', 'Retained_Profits': 'float64', 'Tangible_Assets': 'float64', 'Intangible_Assets': 'float64', 'Total_Fixed_Assets': 'float64', 'Total_Current_Assets': 'float64', 'Trade_Debtors': 'float64', 'Stock': 'float64', 'Cash': 'float64', 'Other_Current_Assets': 'float64', 'Increase_In_Cash': 'float64', 'Mis_Current_Assets': 'float64', 'Total_Assets': 'float64', 'Total_Current_Liabilities': 'float64', 'Trade_Creditors': 'float64', 'Bank_Overdraft': 'float64', 'Other_Short_Term_Fin': 'float64', 'Mis_Current_Liabilities': 'float64', 'Other_Long_Term_Fin': 'float64', 'Total_Long_Term_Liabilities': 'float64', 'Bank_Overdraft_LTL': 'float64', 'Total_Liabilities': 'float64', 'Net_Assets': 'float64', 'Working_Capital': 'float64', 'Paid_up_equity': 'float64', 'PL_Account_Weserve': 'float64', 'Sundry_Weserves': 'float64', 'Revaluation_Weserve': 'float64', 'Shareholder_Funds': 'float64', 'NetWorth': 'float64', 'NetCashflowfromOperations': 'float64', 'NetCashflowbeforeFinancing': 'float64', 'NetCashflowfromFinancing': 'float64', 'Contingent_Liability': 'float64', 'Capital_Employed': 'float64', 'No_Employees': 'float64', 'status': 'float64', 'UPLOAD': 'O'}
AC01_header_names=['AC01', 'REGNUM', 'ACCDAT_start', 'ACCDAT', 'number_of_weeks', 'months', 'currency', 'consolidated', 'acctype', 'Turnover', 'Export', 'Cost_of_Sales', 'Gross_Profit', 'Wages_Salaries', 'Directors_Emoluments', 'Operating_Profits', 'Depreciation', 'Audit_Fees', 'Interest_Payments', 'Pre_Tax_Profit', 'taxation1', 'Profit_After_Tax', 'Dividends_Payable', 'Retained_Profits', 'Tangible_Assets', 'Intangible_Assets', 'Total_Fixed_Assets', 'Total_Current_Assets', 'Trade_Debtors', 'Stock', 'Cash', 'Other_Current_Assets', 'Increase_In_Cash', 'Mis_Current_Assets', 'Total_Assets', 'Total_Current_Liabilities', 'Trade_Creditors', 'Bank_Overdraft', 'Other_Short_Term_Fin', 'Mis_Current_Liabilities', 'Other_Long_Term_Fin', 'Total_Long_Term_Liabilities', 'Bank_Overdraft_LTL', 'Total_Liabilities', 'Net_Assets', 'Working_Capital', 'Paid_up_equity', 'PL_Account_Weserve', 'Sundry_Weserves', 'Revaluation_Weserve', 'Shareholder_Funds', 'NetWorth', 'NetCashflowfromOperations', 'NetCashflowbeforeFinancing', 'NetCashflowfromFinancing', 'Contingent_Liability', 'Capital_Employed', 'No_Employees', 'status', 'UPLOAD']

Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
No: of logical CPU cores available: 8


In [3]:
## Selectting file format
file_format='sav' #(options: csv,sav)

## Selectting user
user='leo' #(user variable options: 'leo','nick')

## Adding filepaths
if(user=='leo'):
    base_dir='C:/Users/cnlp/Research Fellowship/'
elif(user=='nick'):
    base_dir='/Volumes/Pegasus32 R6/CreditSafe 2019 Zipped/'

os.makedirs(base_dir+'UKLTD_Database', exist_ok=True)
dir_list_file=base_dir+'/UKLTD_Scripts/dir_list.txt'
database_file_folder=base_dir+'UKLTD_Database/AC01/'

if (file_format=='csv'): 
    database_file=base_dir+'UKLTD_Database/AC01/AC01.csv'
elif (file_format=='sav'): 
    database_file=base_dir+'UKLTD_Database/AC01/AC01.sav'
else:
    print('File format not supported')

#### Loading database 

In [4]:
## If databse is in .csv format 
if (os.path.exists(database_file_folder)):
    print('Database found.','Reading database! \n') 
    start = time.process_time()
    with ProgressBar():
        df_database = dd.read_parquet(database_file_folder) 
    print('Time taken to read database {}:'.format(database_file_folder),time.process_time() - start,'s')
    database_missing=False
else:
    print('Database missing.  \n')
    exit()
    
print('No: of partitions in the database:',df_database.npartitions)

Database found. Reading database! 

Time taken to read database C:/Users/cnlp/Research Fellowship/UKLTD_Database/AC01/: 0.046875 s
No: of partitions in the database: 4


#### Converting and writing into disk

In [5]:
## Writing converted databse to file
if (file_format=='csv'):
    start = time.process_time()
    with ProgressBar():
        df_database.to_csv(database_file,single_file=True)
    print('Time taken to write:',time.process_time() - start,'s')
elif (file_format=='sav'):
    start = time.process_time()
    with ProgressBar():
        pyreadstat.write_sav(df_database.compute(),database_file)
    print('Time taken to write:',time.process_time() - start,'s')

[########################################] | 100% Completed |  0.1s
Time taken to write: 0.171875 s
